<a href="https://colab.research.google.com/github/baltuna/LT4DH/blob/main/LT4DH_ELTeC_AnalyzingPoS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting part of speech from ELTeC-ENG

Adaptation of a great Colab by Borja Navarro for the LT4DH course in the University of the Basque Country.

This version (to be cleaned) uses English resources in contrast to the Spanish one used by Borja Navarro.

Original reference here:

Borja Navarro Colorado | University of Alicante

In this case, the information about part of speech has not been manually annotated in the corpus. It is necessary first analyze the novels with a NLP system and then extract the linguistic information. The NLP system used is [SpaCy](https://spacy.io/).

The notebook shows:

- how to open a novel from ELTeC in COLAB,
- how to activate SpaCy in COLAB,
- how to analyze the novel with SpaCy, and
- how to extract information about Part of Speec.


## Loading ELTeC-ENG corpus in Colab

In [ ]:
import zipfile

!wget "https://github.com/COST-ELTeC/ELTeC-eng/archive/refs/heads/master.zip" # paste here corpus url

zip_ref = zipfile.ZipFile('master.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts files here (/content/)
zip_ref.close() 
!rm master.zip #Removes ZIP to save space

--2022-03-14 10:53:36--  https://github.com/COST-ELTeC/ELTeC-eng/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/COST-ELTeC/ELTeC-eng/zip/refs/heads/master [following]
--2022-03-14 10:53:36--  https://codeload.github.com/COST-ELTeC/ELTeC-eng/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [      <=>           ]  86.87M  4.10MB/s    in 22s     

2022-03-14 10:53:58 (4.02 MB/s) - ‘master.zip’ saved [91092993]



## SpaCy: download and installing

[SpaCy](https://spacy.io/) is a NLP system. It analyzes part of speech and lemmas, sintax (dependencies) and named entities. 

Three steps:

1. Import SpaCy to Colab
2. Download langauge module (Spanish)
3. Activate module


In [ ]:
import spacy

!python -m spacy download en_core_web_sm #Download English module (the "small" module in this case: "sm").

import en_core_web_sm
nlp_eng = en_core_web_sm.load() #Load Spanish analyzer in "nlp_esp".

     |████████████████████████████████| 12.0 MB 21.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


## Analyzing a novel from ELTeC-ENG

Once we have downloaded the corpus and activated SpaCy, let's analyze one novel.

First, select from the corpus [ELTeC-ENG](https://github.com/COST-ELTeC/ELTeC-eng/tree/master/level1) a novel and copy the file name. Then paste the name in the variable "novela_name". In this example, we will analyze the novel of Gertrudis Gómez de Avellaneda [*Sab*](https://github.com/COST-ELTeC/ELTeC-spa/blob/master/level1/SPA1021_GomezDeAvellaneda_Sab.xml): SPA1021_GomezDeAvellaneda_Sab.xml

In [ ]:
import os
from bs4 import BeautifulSoup

novela_name = "ENG18510_Kingsley.xml" # Put here the name of the file
dir_in = "/content/ELTeC-eng-master/level1/"

novela_text = '' 

print('Analyzing', novela_name)

ficheroEntrada = dir_in + novela_name
with open(ficheroEntrada, 'r') as tei: #Opens the file
  print("Opening the file and extracting text")
  soup = BeautifulSoup(tei, 'xml') #Parse the XML
  capitulos = soup.find_all(type="chapter") #Only chapters are taking into account. No letters (To Do)
  for cap in capitulos:
    parrafos = cap.find_all('p') #Extract all paragraphs of each chapter
    for parrafo in parrafos:
      #print(parrafo.text)
      novela_text+=parrafo.text+'\n'

print('Analyzing PoS and lemmas')
analisis = nlp_eng(novela_text) #Here the novel is analyzed with SpaCy. All the analysis is stored in "analisis" variable.
print('Done!')

Analyzing ENG18510_Kingsley.xml
Opening the file and extracting text
Analyzing PoS and lemmas
Done!


Now all the analysis is stored in "analisis" variable. It only remains to iterate over the variable and extract the information: in this case, part of speech. How to extract information about syntax, named entities, etc. see [SpaCy 101](https://spacy.io/usage/spacy-101)

In [ ]:
NVA = '\tNovel\tNouns\tVerbs\tAdjectives\tUnique_nouns\tUnique_verbs\tUnique_adjs\n' #

nom_novela = 'Yeats'
#nom_novela = novela_name

nouns=[]
verbs=[]
adjs=[]

unique_noun=[]
unique_verb=[]
unique_adj=[]

for token in analisis: 
  if token.pos_ == 'NOUN':
    nouns.append(token.lemma_) #
    if token.lemma not in unique_noun:
       unique_noun.append(token.lemma)
  elif token.pos_ == 'VERB':
    verbs.append(token.lemma_) #
    if token.lemma not in unique_verb:
       unique_verb.append(token.lemma)
  elif token.pos_ == 'ADJ':
    adjs.append(token.lemma_) #
    if token.lemma not in unique_adj:
       unique_adj.append(token.lemma)


num_nouns = str(len(nouns))
num_verbs = str(len(verbs))
num_adjs = str(len(adjs))
num_uninouns = str (len(unique_noun))
num_univerbs = str (len(unique_verb))
num_uniadjs = str (len(unique_adj))

NVA += '\t'+nom_novela+'\t'+num_nouns+'\t'+num_verbs+'\t'+num_adjs+'\t'+num_uninouns+'\t'+num_univerbs+'\t'+num_uniadjs+'\n' #

print(NVA)

#salida = open('analisis_soloNombres.txt', 'w') #Crea fichero, etc.
#salida.write(corpus_soloNombres)
#salida.close()
#files.download('analisis_soloNombres.txt')




	Novel	Nouns	Verbs	Adjectives	Unique_nouns	Unique_verbs	Unique_adjs
	Yeats	15409	12903	6691	3548	1662	1393

